In [1]:
import numpy as np
import main

In [2]:
M_ls = []

In [3]:
rod_depth_1 = 80
rod_depth_2 = 0
total_height = 160
# Main material
M = main.material_FDM(mat_id=0)
M.set_XS('XS_scatter',[[0,0.017555500],[0,0]])
M.set_XS('Diffusion_coef',[1.423913000,0.356306000])
M.set_XS('XS_absorption',[0.010402060,0.087662170])
M.set_XS('XS_nu_fission',[0.006477691,0.112732800])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

# Control rod 1
M = main.material_FDM(mat_id=1)
M.set_XS('XS_scatter',[[0,0.017555500],[0,0]])
M.set_XS('Diffusion_coef',[1.423913000,0.356306000])
M.set_XS('XS_absorption',[(0.010952060*rod_depth_1+0.010402060*(total_height-rod_depth_1))/total_height,(0.091462170*rod_depth_1+0.087662170*(total_height-rod_depth_1))/total_height])
M.set_XS('XS_nu_fission',[0.006477691,0.112732800])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

M = main.material_FDM(mat_id=2)
M.set_XS('XS_scatter',[[0,0.01717768],[0,0]])
M.set_XS('Diffusion_coef',[1.425611000,0.350574000])
M.set_XS('XS_absorption',[0.010992630,0.099256340])
M.set_XS('XS_nu_fission',[0.007503284,0.137800400])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

M = main.material_FDM(mat_id=3)
M.set_XS('XS_scatter',[[0,0.02759693],[0,0]])
M.set_XS('Diffusion_coef',[1.634227000,0.264002000])
M.set_XS('XS_absorption',[0.002660573,0.049363510])
M.set_XS('XS_nu_fission',[0.0,0.0])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

# Control rod 2
M = main.material_FDM(mat_id=4)
M.set_XS('XS_scatter',[[0,0.017555500],[0,0]])
M.set_XS('Diffusion_coef',[1.423913000,0.356306000])
# M.set_XS('XS_absorption',[0.010952060,0.091462170])
M.set_XS('XS_absorption',[(0.010952060*rod_depth_2+0.010402060*(total_height-rod_depth_2))/total_height,(0.091462170*rod_depth_2+0.087662170*(total_height-rod_depth_2))/total_height])
M.set_XS('XS_nu_fission',[0.006477691,0.112732800])
M.set_XS('XS_fission_spectrum',[1.0,0.0])
M_ls.append(M)

In [4]:
precursor_1 = main.precursor_FDM(delay_group_num=6)
precursor_1.set_beta([0.0002470,0.0013845,0.0012220,0.0026455,0.0008320,0.0001690])
precursor_1.set_lambda([0.0127,0.0317,0.1150,0.3110,1.4000,3.8700])

In [5]:
geo = main.geometry_FDM(x_block=6,y_block=6)
layout = np.array([[5,1,1,2,3,4],[1,1,1,1,3,4],[1,1,5,1,3,4],[2,1,1,3,3,4],[3,3,3,3,4,4],[4,4,4,4,4,4]],dtype=np.int32)
layout = layout-1

geo.set_block_mat_by_array(layout)

geo.set_block_size(x_size=[10,20,20,20,20,20],y_size=[10,20,20,20,20,20])
geo.set_discretized_num(x_dim=[10,20,20,20,20,20],y_dim=[10,20,20,20,20,20])
geo.check_blocks()

Material layout:
[[4 0 0 1 2 3]
 [0 0 0 0 2 3]
 [0 0 4 0 2 3]
 [1 0 0 2 2 3]
 [2 2 2 2 3 3]
 [3 3 3 3 3 3]]
Block size:
x:[10, 20, 20, 20, 20, 20]
y:[10, 20, 20, 20, 20, 20]
Discretization number:
x:[10, 20, 20, 20, 20, 20]
y:[10, 20, 20, 20, 20, 20]


In [6]:
case = main.solver_FDM(folder_name='LMW_2D',group_num=2)
case.add_material(M_ls)
case.set_geometry(geo)
case.set_geometry(geo,transient_mode=True)
case.set_precursor(precursor_1)
case.set_neutron_velocity([1.25e7,2.5e5])

In [7]:
# time_steps = np.array([0,0.005])
time_steps = np.linspace(0.0, 0.0, num=1)
# time_steps = np.array([0])
for time_index,time_step in enumerate(time_steps):
    print('Time={} (s)'.format(time_steps[time_index]))
    if time_index==0:
        # To get initial flux distribution
        case.solve_source_iter_correct(max_iter=100,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        # case.solve_source_adjoint_iter_correct(max_iter=50,k_tolerance=1e-5,flux_tolerance=1e-4,initial_k=1.0)
        # case.initial_dynamics(transient_algorithm='Implicit_Euler')
        case.initial_dynamics(time_steps=time_steps,transient_algorithm='SCM',vtk_save=False)
    else:
        # Update material
        # Control rod 1
        if time_step <=26.5:
            rod_depth_1 = 80-time_step*80/26.5
            # rod_depth_1 = 80
        else:
            rod_depth_1 = 0.0
        M_ls[1].set_XS('XS_absorption',[(0.010952060*rod_depth_1+0.010402060*(total_height-rod_depth_1))/total_height,(0.091462170*rod_depth_1+0.087662170*(total_height-rod_depth_1))/total_height])
        case.update_material(1,M_ls[1],mat_type='real')
        # Control rod 2
        if time_step <=47.5 and time_step>=7.5:
            rod_depth_2 = 3*(time_step-7.5)# Insert control rod
            # rod_depth_2 = 0 # Insert control rod
        elif time_step<7.5:
            rod_depth_2 = 0
        else:
            rod_depth_2 = 120
        M_ls[4].set_XS('XS_absorption',[(0.010952060*rod_depth_2+0.010402060*(total_height-rod_depth_2))/total_height,(0.091462170*rod_depth_2+0.087662170*(total_height-rod_depth_2))/total_height])
        case.update_material(4,M_ls[4],mat_type='real')
        # Solve transient problem with SCM
        time_interval = time_steps[time_index]-time_steps[time_index-1]
        case.solve_transient_SCM_2(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-6)
        # case.solve_transient_Implicit_Euler(time_index,time_interval,max_iter=200,k_tolerance=1e-5,flux_tolerance=1e-4)
        # case.solve_transient_SCM_PKE_predictor(time_index,time_interval,max_iter=100,k_tolerance=1e-6,flux_tolerance=1e-4,k_outer_tolerance=1e-6)

Time=0.0 (s)
D:\Users\85136\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
####################################
Timestep 0 calculation begins
c:\Users\85136\OneDrive\Code\FDM\main.py:1724: RuntimeWarning: invalid value encountered in true_divide
  disp = abs((dist-dist_last)/dist)
Iteration 1: Eigenvalue k = 0.9797126730807953
Iteration 2: Eigenvalue k = 0.9921706214596219
Iteration 3: Eigenvalue k = 0.9986787248450786
Iteration 4: Eigenvalue k = 1.0025859323790205
Iteration 5: Eigenvalue k = 1.005176633312608
Iteration 6: Eigenvalue k = 1.0070207318067719
Iteration 7: Eigenvalue k = 1.0084040783482373
Iteration 8: Eigenvalue k = 1.0094838471802279
Iteration 9: Eigenvalue k = 1.0103528790322367
Iteration 10: Eigenvalue k = 1.0110692726599224
Iteration 11: Eigenvalue k = 1.0116711564758765
Iteration 12: Eigenvalue k = 1.012184570100425
Iter